# attention中的query_key_value基本概念


## 相关代码
### repeat_kv

In [3]:
import torch
def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    '''这个函数通常被用于在某个特定的维度上复制张量，这样可以创建出具有更高维度的数据。
    x : 输入的张量
    n_rep : 复制次数
    '''
    # bs : batch size
    # slen : sequence length
    # n_kv_heads : number of key/value heads
    # head_dim : head dimension
    bs, slen, n_kv_heads, head_dim = x.shape
    
    # 如果n_rep等于1，那就没有必要复制张量，函数直接返回输入张量x。
    if n_rep == 1:
        return x 
    
    # x[:, :, :, None, :]  添加新维度
    # 通过在第四个维度位置插入一个新的维度（使用None或np.newaxis）来增加输入张量x的维度。
    
    # .expand(bs, slen, n_kv_heads, n_rep, head_dim) 张量扩展
    # 使用expand方法来扩展张量的新维度，从而复制该维度。n_rep参数控制了新维度的大小，即复制的次数。
    
    # .reshape(bs, slen, n_kv_heads * n_rep, head_dim) 重塑张量
    # 使用reshape方法将扩展后的张量重塑为最终的形状，这里通过乘以n_rep将 _kv_heads和新添加的维度合并为一个维度，从而得到最终的形状。
    return (
        x[:, :, :, None, :]
        .expand(bs, slen, n_kv_heads, n_rep, head_dim)
        .reshape(bs, slen, n_kv_heads * n_rep, head_dim)
    )

In [4]:
# 1. 准备数据

# 输入数据
x = torch.tensor([
    [[1, 2], [3, 4]],
    [[5, 6], [7, 8]]
])

# 复制次数
n_rep = 3

print(x.shape)

# 2. 调用上面定义的函数

# 为 x 增加一个维度，匹配四个维度: bs, slen, n_kv_heads, head_dim
x = x.unsqueeze(-1) # x shape : (2, 2, 2, 1)

# 调用函数
output_tensor = repeat_kv(x, n_rep)

print(output_tensor.shape)

torch.Size([2, 2, 2])
torch.Size([2, 2, 6, 1])


## 计算Query/Key/Value向量原理

在自注意力机制中，输入是一个序列的向量表示，每个向量都要被转换为三个不同的向量：Query（查询）向量、Key（键）向量和Value（值）向量。这三个向量是通过线性变换得到的，即通过与权重矩阵相乘来计算。  


对于 Query 向量的计算，公式如下：  
$Query$ = $X \cdot W_Q$   
其中: 
- $X$是输入向量，其形状为: (batch_size, seq_len, dim)  
- $W_Q$ 是 Query向量的权重矩阵，其形状为 (dim, n_head $\cdot$ head_dim)  
- $Query$是计算得到Query向量，其形状为 (batch_size, seq_len, n_head $\cdot$ head_dim)   

$Key$ 和 $Value$ 向量的计算原理与上一致。  

$Key = X \cdot W_k$

$Value = X \cdot W_v$  

在Transformer模型及其变体中，自注意力机制的核心是通过Query、Key和Value三种向量来计算。每种向量都可以有多个“头”，每个头都有其独特的权重矩阵，从而可以捕获输入序列中的不同依赖关系。这种设计使得模型可以从多个不同的子空间中学习输入序列的特征。  

**（1）注意力头（Attention Heads）**  
注意力头是自注意力机制的基本组成单元，每个注意力头都有一套独立的权重矩阵用于计算Query、Key和Value向量。通过这种设计，每个注意力头可以从不同的角度或者说是在不同的子空间中捕获序列的依赖关系。通常，模型会有多个注意力头，它们并行工作并学习输入序列的不同特征。`args.n_heads` 就是表示这种总的注意力头的数量。

**（2）键值头（Key/Value Heads）**  
在某些特定的设计或优化中，可能会对Key和Value向量的头数量进行特殊的设置，这就是所谓的键值头。这种设计可能是基于某些特定的优化目标，比如减少计算量、适应特定的硬件配置或者实现某种特定的算法优化。`self.n_kv_heads` 就是表示这种键值头的数量。


**（3）区别**  
    **（3.1）功能和目标**    
    ① 注意力头是为了让模型能够从多个不同的角度学习输入序列的特征，每个头都可以捕获不同的依赖关系。  
    ② 键值头可能是基于某些特定的优化目标而设计的，它可能涉及到模型的并行计算、资源分配或者其他的优化目标。   
    **（3.2）数量的设置**  
    ① 注意力头的数量通常是固定的，它是模型结构的一部分，与模型的其他参数一起进行训练。  
    ② 键值头的数量可能是动态的，它可能根据特定的优化目标或者运行时的条件来进行调整。    
    **（3.3）权重矩阵**  
    ① 注意力头每个头都有独立的权重矩阵用于计算Query、Key和Value向量。  
    ② 键值头可能共享某些权重矩阵，或者有特定的权重矩阵用于计算Key和Value向量。

## KV Cache 算法原理
**（1）自注意力机制概述**  
在Transformer模型中，自注意力机制是核心组成部分。其基本思想是允许模型在处理一个元素（例如一个单词）时，同时考虑到序列中的其他元素。这是通过计算 Query (Q)，Key (K)，和 Value (V) 来实现的：
- Query (Q)：当前要处理的元素。
- Key (K)：序列中所有元素的表示，用于匹配Query。
- Value (V)：序列中所有元素的另一种表示，一旦Query与Key匹配，相应的Value会被用来计算输出。  

**（2）KV Cache（Key-Value Cache）**  
① KV Cache在Transformer模型的自注意力机制中是一种重要的优化方法，特别是在处理长序列数据时。  
② 在基本的Transformer模型中，每个序列位置的Query向量需要与所有位置的Key向量进行点积运算以计算注意力权重。    
③ 在处理长序列时，这种全序列的计算会非常耗时和耗资源。为了优化这种计算，可以使用KV Cache来保存前面步骤计算过的Key和Value向量，从而提高效率和减少计算量。  

**（3）KV Cache 缓存过程**  

通过一个简化的示例来说明KV Cache的工作原理：

假设有一个长序列x，长度为10，即x = [x1, x2, ..., x10]，我们希望计算每个位置的Query向量与所有位置的Key向量的点积。在没有优化的情况下，需要对每个位置进行10次计算，总共需要100次计算。


- **1. 初始化 KV Cache**  
首先，初始化空的KV Cache来存储Key和Value向量。假设每个向量的维度为2，我们可以创建两个大小为(10, 2)的零张量来初始化Key和Value的缓存。  
```shell
cache_k = torch.zeros((10, 2))  # 用于存储Key向量
cache_v = torch.zeros((10, 2))  # 用于存储Value向量
```

- **2. 分批处理序列**  
为了减少每个位置所需的计算次数，可以将长序列分成若干部分，每部分包含一定数量的位置。例如，将序列x分成两部分，每部分包含5个位置。  

- **3. 处理第一部分**  
在处理序列的第一部分时，为每个位置计算Key向量和Value向量，并将它们存储在KV Cache中。  
```shell
# 假设第一部分的Key向量和Value向量为：
keys_1 = torch.randn((5, 2))  # 随机生成
values_1 = torch.randn((5, 2))  # 随机生成

# 更新 KV Cache
cache_k[:5, :] = keys_1
cache_v[:5, :] = values_1
```  

- **4. 处理第二部分**  
在处理序列的第二部分时，为每个位置计算Key向量和Value向量，同时，将新计算的Key向量与KV Cache中的Key向量合并。  
```shell
# 假设第二部分的Key向量和Value向量为：
keys_2 = torch.randn((5, 2))  # 随机生成
values_2 = torch.randn((5, 2))  # 随机生成

# 更新KV Cache
cache_k[5:, :] = keys_2
cache_v[5:, :] = values_2

# 合并所有Key向量和Value向量
all_keys = cache_k
all_values = cache_v
```  

- **5.计算点积**  
现在，为每个位置计算Query向量。然后，使用Query向量与合并后的所有Key向量计算点积。由于Key向量已经计算并存储在KV Cache中，因此只需为每个位置计算一次Query向量，并与所有Key向量计算点积。  
```shell
for i in range(10):
    q_i = ...  # 计算位置i的Query向量
    dot_products = q_i @ all_keys.T  # 计算点积

```
通过上述优化过程，我们减少了对每个位置的计算次数，从而降低了计算复杂度，并利用了KV Cache来存储和重用先前计算的Key和Value向量。  

**（4）KV 缓存的优势**  

1. 减少计算量：

由于不需要对已处理的序列部分重新计算 K 和 V，因此大大减少了总体计算量。

2. 实时序列处理：

在实时应用（如在线翻译）中，KV 缓存使得模型能够即时处理输入，同时保持对之前输入的记忆。

3. 长序列处理：

对于长序列，直接处理整个序列可能会导致计算资源不足。KV 缓存通过分步处理，使得模型能够有效地处理长序列。

## Self-Attention，Encoder-Decoder Attention
**（1）自注意力机制 (Self-Attention)**  
- ① 上下文捕捉:   
自注意力机制能够捕捉输入序列中的长距离依赖关系。通过计算序列中每个元素与其他所有元素之间的注意力权重，模型可以了解哪些元素是相关的，从而捕捉到不同元素之间的依赖关系。  

- ② 并行计算:   
由于自注意力机制可以同时处理序列中的所有元素，它支持并行计算，这使得模型能够快速处理长序列。 

- ③ 应用于编码器和解码器:   
自注意力机制被应用于Transformer的编码器和解码器中，帮助模型捕捉源语言和目标语言内部的上下文信息。

 
 
**（2）编码器-解码器注意力机制 (Encoder-Decoder Attention)**  
- ① 源-目标语言交互:   
编码器-解码器注意力机制主要用于在解码阶段，使目标语言的解码过程能够参考源语言的上下文信息。通过计算目标语言中每个位置与源语言中所有位置之间的注意力权重，模型可以根据源语言的上下文来生成目标语言的输出。  

- ② 上下文引导的翻译:   
通过编码器-解码器注意力机制，解码器可以在生成每个新单词时都考虑源语言的上下文，从而实现更准确的翻译。  

- ③ 注意力可视化:   
编码器-解码器注意力机制还提供了一种可视化注意力权重的方式，从而可以直观地理解模型是如何将源语言和目标语言中的不同部分对应起来的。


**（3）两种注意力机制的融合**  
这两种注意力机制的融合主要是通过层的堆叠和残差连接来实现的。下面详细描述了这两种注意力机制是如何在解码阶段融合的：  

**<font color=red>（3.1）层的堆叠:</font>**  

> 解码器的每一层都包含一个自注意力子层和一个编码器-解码器注意力子层。这两个子层是顺序执行的，即先执行自注意力子层，然后执行编码器-解码器注意力子层。通过这种方式，解码器的每一层都能获得目标语言的内部上下文（通过自注意力机制）和源语言的上下文（通过编码器-解码器注意力机制）。

**<font color=red>（3.2）残差连接:</font>**  

> 在每个子层中，都有一个残差连接，它将子层的输入添加到子层的输出中。这样，每个子层的输出都包含了原始的输入信息和子层处理过的信息。残差连接有助于保持信息流，并避免在深层网络中出现梯度消失问题。

**<font color=red>（3.3）具体的融合过程如下：</font>**  

**① 输入:**   
解码器的每一层接收两个输入：一个是来自上一层的输出（或者在第一层时是来自嵌入层的输出），另一个是编码器的最终输出。

**② 自注意力子层:**   
首先，输入数据通过自注意力子层，该子层计算目标语言已翻译部分的内部上下文。
通过残差连接，自注意力子层的输出包含了原始的输入信息和自注意力处理过的信息。

**③ 编码器-解码器注意力子层:**  
然后，自注意力子层的输出和编码器的输出一起传递到编码器-解码器注意力子层。
在这个子层中，模型计算源语言和目标语言之间的注意力权重，并生成一个上下文向量，该向量包含了源语言的上下文信息。
通过残差连接，编码器-解码器注意力子层的输出包含了自注意力子层的输出和编码器-解码器注意力处理过的信息。

**④ 前馈神经网络:**  
最后，编码器-解码器注意力子层的输出传递给一个前馈神经网络，该网络进一步处理数据，为下一层或最终的输出做准备。

## mask 原理
**（1）什么是Mask（掩码）**  
在深度学习模型，特别是序列处理模型如Transformer中，掩码（Mask）是一个重要的概念，它用于屏蔽序列中某些位置的信息，保证模型在处理时不会“看到”这些位置的信息。  

**（2）Mask的作用**  
a. 防止信息泄露   
在序列预测任务中，如语言模型中的下一个词预测，掩码确保模型在预测某个时间点的输出时，不会看到未来的信息。这种掩码通常称为“未来信息掩码”或“因果掩码”。

b. 处理不同长度的序列   
在处理长度不一的序列时，较短的序列会被填充（Padding）以匹配最长序列的长度。掩码在这里用来屏蔽这些填充值，以确保它们不会影响模型的学习。

**（3）Mask的实现方式**  
在实践中，mask通常通过与数据序列相乘或在注意力分数上应用加法操作来实现。对于要屏蔽的位置，mask会将注意力分数加上一个非常大的数（例如-∞），使得经过softmax后，这些位置的权重接近于零。

**（4）Mask对训练和推理的影响**  
a. 训练阶段   
在模型训练时，掩码帮助模型更加专注于关键信息，提高了训练的效率和准确性。特别是在处理不同长度的序列时，掩码确保了模型不会受到填充值的干扰。

b. 推理阶段  
在模型推理（如生成文本）时，掩码保证了生成的连贯性和逻辑性，尤其是在基于上下文的生成任务中。

**（5）举例说明**  
假设我们有一个序列 ["I", "love", "AI", "."]，我们想要用Transformer模型来处理这个序列。   
在自注意力机制的计算过程中，每个单词都需要与序列中的所有单词计算注意力分数。但在某些情况下，我们可能不想让某些单词“看到”序列中的其他单词。例如，在解码阶段，为了保证模型按照从左到右的顺序生成单词，我们不希望模型能看到未来的单词。  

- **第1步：创建掩码**  
为了实现这个目的，我们可以创建一个掩码。假设我们当前正在处理单词 "love"，我们希望模型不能看到它右边的单词 "AI" 和 "."。我们可以创建以下掩码：  
<img src="../images/08.png"/>    

这个掩码是一个 4x4 的矩阵，其中 0 表示模型可以“看到”对应位置的单词，-inf（负无穷）表示模型不能“看到”对应位置的单词。  

- **第2步：应用掩码**  
在计算注意力分数时，我们通常会计算 Query 和 Key 的点积，然后通过 Softmax 函数将分数转换为概率分布。在应用 Softmax 函数之前，我们将掩码加到注意力分数上：  
```shell
attention_scores = dot_product(Queries, Keys) + mask
```  
由于 mask 中的 -inf 会使得对应位置的注意力分数变为负无穷，经过 Softmax 函数处理后，这些位置的注意力权重将接近于 0，从而实现了屏蔽的效果。

### 核心源码

In [ ]:
from typing import Optional
import torch.nn as nn

class Attention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        # n_kv_heads : 该变量表示键（Key）和值（Value）的头的数量
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        # 获取模型并行的world size（也就是并行执行模型的设备数量）
        model_parallel_size = fs_init.get_model_parallel_world_size()
        # 表示本地（即每个设备上）的头的数量，计算方式是总的头的数量除以模型并行的world size
        self.n_local_heads = args.n_heads // model_parallel_size
        # 表示本地的键和值的头的数量，计算方式是总的键和值的头的数量除以模型并行的world size
        self.n_local_kv_heads = self.n_kv_heads // model_parallel_size
        # 表示每个本地键和值的头需要重复的次数，计算方式是本地头的数量除以本地键和值的头的数量
        self.n_rep = self.n_local_heads // self.n_local_kv_heads
        # 表示每个头的维度，计算方式是模型的维度除以头的数量
        self.head_dim = args.dim // args.n_heads
        
        # 用于计算Query（查询）向量
        self.wq = ColumnParallelLinear(
            args.dim, # 输入数据的维度
            self.n_heads * self.head_dim, # 输出的维度，等于注意力头的数量乘以每个头的维度。
            bias = False, # 表示这个线性层不包含偏置项
            gather_output=False, # 表示输出不会在所有的设备上收集，而是分布在各个设备上。这样可以减少通信开销，提升模型并行的效率。
            init_method=lambda x: x, # 初始化权重矩阵的函数，这里使用的是一个恒等函数，表示权重矩阵在初始化后不会被改变。
        )
        
        # 用于计算 Key（键）向量
        self.wk = ColumnParallelLinear(
            args.dim,
            self.n_kv_heads * self.head_dim,
            bias = False, 
            gather_output = False,
            init_method = lambda x: x,
        )
        
        # 用于计算 Value（值）向量
        self.wv = ColumnParallelLinear(
            args.dim,
            self.n_kv_heads * self.head_dim,
            bias=False,
            gather_output=False,
            init_method=lambda x: x,
        )
        
        # 用于计算 Wo（output)
        self.wo = RowParallelLinear(
            args.n_heads * self.head_dim, # 输入的维度，等于注意力头的数量乘以每个头的维度。
            args.dim, # 输出数据的维度，通常表示词嵌入或输入向量的维度
            bias=False,
            input_is_parallel=True, # 表示输入数据已经分布在各个设备上，在模型并行（model parallel）的设置中
            init_method=lambda x: x,
        )
        
        '''为什么 self.wq 和 self.wk, self.wv 使用ColumnParallelLinear，而 self.wo 使用 RowParallelLinear ？
        1. 模型结构的设计
           self.wq和self.wk,self.wv是用于计算Query和Key向量的，而self.wo是用于计算输出向量的。
           在自注意力机制中，Query和Key向量的计算是为了得到注意力权重，而输出向量的计算是基于得到的注意力权重和Value向量。
           
        2. 优化计算效率和内存消耗
           ColumnParallelLinear和RowParallelLinear可能是为了在不同的计算阶段优化计算效率和内存消耗。
           例如，在计算Query和Key向量时，可能更关心计算效率，而在计算输出向量时，可能更关心内存消耗。
        '''
        
        
        # self.cache_k是一个缓存矩阵，它用于存储键(k)向量。在Transformer模型，特别是在处理自然语言处理任务中的解码阶段，
        # 键向量被缓存起来可以提高计算效率。这是因为在解码阶段，模型通常是一个单词一个单词地生成输出序列，
        # 而每个新生成的单词都需要与所有先前生成的单词的键向量进行交互（通过计算注意力分数）。
        # 因此，将先前生成的单词的键向量存储起来可以避免重复计算。
        self.cache_k = torch.zeros(
            (
                args.max_batch_size, # 最大的批处理大小
                args.max_seq_len, # 序列的最大长度
                self.n_local_kv_heads, # 在模型并行设置中，每个设备上的键/值头的数量
                self.head_dim, # 每个头的维度
            )
        ).cuda()
        
        
        self.cache_v = torch.zeros(
            (
                args.max_batch_size,
                args.max_seq_len,
                self.n_local_kv_heads,
                self.head_dim,
            )
        ).cuda()
        
        def forward(
            self,
            x: torch.Tensor, # 输入的张量，通常包含了一个batch的序列数据
            start_pos: int, # 在处理非常长的序列，这时就需要分批处理，start_pos就是每一批处理开始的位置
            freqs_cis: torch.Tensor, # 旋转位置编码的频率张量，用于执行旋转位置编码。
            mask: Optional[torch.Tensor], # 用于遮盖某些位置，如果序列长度不一，就需要添加padding，并使用mask遮盖padding的位置
        ):
            bsz, seqlen, _ = x.shape
            # 输入x会分别通过三个不同的线性变换（也就是权重矩阵 wq、wk 和 wv），
            # 这三个线性变换分别对应于查询（query）、键（key）和值（value）。
            xq, xk, xv = self.wq(x), self.wk(x), self.wv(x)
            
            # 将xq、xk 和xv重塑为四维张量，其中第三维表示头的数量（对于xq是本地头的数量，对于xk和xv是键/值头的数量），
            # 第四维表示每个头的维度，这样做的目的是为了后续的并行计算和注意力打分。
            xq = xq.view(bsz, seqlen, self.n_local_heads, self.head_dim)
            xk = xk.view(bsz, seqlen, self.n_local_kv_heads, self.head_dim)
            xv = xv.view(bsz, seqlen, self.n_local_kv_heads, self.head_dim)
            
            # 应用ROPE，这是一种新型的位置编码方式，可以解决Transformer模型对于输入序列长度的限制问题。
            # 具体来说，ROPE将位置信息以旋转的方式编入到序列的每个位置中，旋转的角度与位置有关，
            # 从而使得模型可以区分不同的位置。
            xq, xk = apply_rotary_emb(xq, xk, freqs_cis=freqs_cis)
            
            # 移动到xq所在的device上
            self.cache_k = self.cache_k.to(xq)
            self.cache_v = self.cache_v.to(xq)
            
            # 注意力机制在进行计算时，会使用到全部的历史信息（即所有的Key和Value），因此需要将新的xk和xv添加到缓存中。
            # 这里使用的是覆盖的方式，也就是说，对于当前批次bsz的数据，从位置start_pos开始，长度为seqlen的位置，
            # 都会被新的xk和xv覆盖。这样，当处理下一批数据时，缓存中就包含了所有已经处理过的历史信息，
            # 从而可以进行全历史范围的注意力计算。
            self.cache_k[:bsz, start_pos : start_pos + seqlen] = xk
            self.cache_v[:bsz, start_pos : start_pos + seqlen] = xv
            
            # 先取出存储在self.cache_k和self.cache_v中的键（keys）和值（values）
            # 这些键和值包含了从序列开始到当前处理位置（start_pos + seqlen）的所有信息
            keys = self.cache_k[:bsz, : start_pos + seqlen]
            values = self.cache_v[:bsz, : start_pos + seqlen]
            
            # 重复键和值以匹配头的数量
            # 当键/值的头数量（self.n_kv_heads）小于总的头数量（self.n_heads）时，
            # 每个键/值需要被复制多次（self.n_rep 次）以匹配头的数量。
            # 通过这种方式，每个头可以对应一个独立的键/值，从而进行并行处理。
            keys = repeat_kv(keys, self.n_rep)
            values = repeat_kv(values, self.n_rep)
            
            # 下面代码实现了注意力机制中的核心计算，即计算query和key的点积，得到每个token对其他所有token的注意力得分。
            xq = xq.transpose(1, 2)
            keys = keys.transpose(1, 2)
            values = values.transpose(1, 2)
            
            # 使用torch.matmul函数计xq和keys的乘积，这里keys需要在最后两个维度之间进行转置（transpose(2, 3)），
            # 因为在进行矩阵乘法时，我们需要最后一个维度和倒数第二个维度相匹配。
            # 也就是说，我们在进行query和key的点积计算时，使用的是每个头部分的query和key。
            # 将得到的scores除以sqrt(self.head_dim)，这是一种常见的缩放操作，用于缓解点积可能导致的梯度消失或梯度爆炸问题。
            scores = torch.matmul(xq, keys.transpose(2, 3)) / math.sqrt(self.head_dim)
            
            # 如果提供了mask，将会将mask加到scores上，以将不应被关注的位置设置为负无穷，
            # 从而在softmax计算时，被设置为接近0的权重。这个操作主要用在自注意力机制中，例如防止解码器看到未来的token。
            if mask is not None:
                scores = scores + mask # (bs, n_local_heads, seqlen, cache_len + seqlen)
            
            # 使用F.softmax函数对scores进行处理，使得每一行的和为1。
            # 这就是我们通常说的“attention weights”，也就是我们在这一步要对每个位置赋予的重要性。
            scores = F.softmax(scores.float(), dim=-1).type_as(xq)
            
            # 将softmax的输出（即注意力权重）和values进行乘法运算，从而得到每个位置的加权平均。
            # 这就是我们通常说的“context”，因为这个输出向量包含了输入序列所有位置的信息，而每个位置的贡献是由其对应的注意力权重决定的。
            output = torch.matmul(scores, values) # (bs, n_local_heads, seqlen, head_dim)
            
            # 将output的形状由[bsz, n_local_heads, seqlen, head_dim]调整为[bsz, seqlen, n_local_heads * head_dim]
            # .contiguous(): 这是一个PyTorch中的特殊操作，其目的是使得张量在内存中连续排布。
            output = output.transpose(1, 2).contiguous().view(bsz, seqlen, -1)
            
            # 使用定义的线性变换 self.wo 对输出张量进行线性变换，并返回结果。
            return self.wo(output)